In [547]:
import numpy as np
import boto3
import pandas as pd
import math
import matplotlib.pyplot as plt

t_max = 1e-6
micro_seconds = 1e-6
micro_meter = 1e-6
megahertz = 1e+6

In [3]:
from braket.tracking import Tracker
tracker = Tracker().start()

In [572]:
tracker.quantum_tasks_statistics()

{<_QuEra.Aquila: 'arn:aws:braket:us-east-1::device/qpu/quera/Aquila'>: {'shots': 15000,
  'tasks': {'QUEUED': 15}}}

In [ ]:
# tracker.qpu_tasks_cost()

In [762]:
dataSet = "FashionMNIST"
digitClass = 1
noiseChoice = 2
rabiPulse = "trapezoidPulse"
detunePulse = "gaussianPulse"
folder = "pythonInput/"+dataSet+"_"+str(digitClass)+"/" + rabiPulse + detunePulse + "/"


In [763]:
# cosine pulse
def cosinePulse(time, pulse_parameter, noise):
    return ((pulse_parameter/2) * -math.cos(2 * 2 * math.pi * time) * noise) + (pulse_parameter/2)

# exponential pulse
def exponentialPulse(time, pulse_parameter, noise):
    peakValue = pulse_parameter * noise
    peakTime = 0.2 # needs to be changed in scale

    if time <= peakTime:
        val = (peakValue/peakTime) * time
    else:
        val = math.exp(-(time-peakTime)/0.2) * peakValue
    
    return val

# linear pulse
def linearPulse(time, pulse_parameter, noise):
    start = 0
    middle = (-2*math.pi) * 13 * noise

    if time <= 0.1:
        val = (middle/0.1) * time
    elif (time > 0.1) & (time <= 0.2):
        val = middle
    elif (time > 0.2) & (time < 0.8):
        val = (((pulse_parameter - middle)/0.6) * (time - 0.2)) + middle
    elif (time >= 0.8) & (time <= 0.9):
         val = pulse_parameter
    elif time > 0.9:
        val = pulse_parameter + ((start-pulse_parameter)/0.1) * (time-0.9)

    return val

# triangle pulse
def trianglePulse(time, pulse_parameter, noise):
    val = 0

    if time < 0.5:
        val = (pulse_parameter/0.5) * time
    else:
        val = ((-pulse_parameter/0.5) * time) + (2 * pulse_parameter)

    return val

# exponentialDecayPulse
def exponentialDecayPulse(time, pulse_parameter, noise):
    peakValue = -120.0 * noise
    peakTime = 0.2 # needs to be changed in scale

    if time <= peakTime:
        val = (peakValue/peakTime) * time
    else:
        val = peakValue * math.exp(-(time-peakTime)*pulse_parameter**2)
    return val

# guassian Pulse
def guassianPulse(time, pulse_parameter, noise):
    return pulse_parameter * math.exp(-((time-0.5)**2)/(0.01*noise)) # order of events

# trapezoid Pulse
def trapezoidPulse(time, pulse_parameter, noise):
    risingTime = 0.1 * noise # needs to be changed for scale

    if time <= risingTime:
        val = (pulse_parameter/risingTime) * time
    elif (time > risingTime) & (time < (1.0-risingTime)):
        val = pulse_parameter
    else:
        val = -(pulse_parameter/risingTime) * (time - 1.0)


    return val

In [764]:
pulse_dictionary = {'sinPulse':cosinePulse, 'expPulse':exponentialPulse, 'linearPulse':linearPulse,
                    'trianglePulse':trianglePulse, 'expDecayPulse':exponentialDecayPulse, 'gaussianPulse':guassianPulse,
                    'trapezoidPulse':trapezoidPulse}


In [765]:
from braket.timings.time_series import TimeSeries

def getTimeSeries(pulse_shape_data, noise_data, pulse_parameter_data, value_count, start_time=0, end_time=1):
    
    times = np.linspace(start=start_time, stop=end_time, num=value_count, endpoint=True)
         
    results = {'rabi_ts':None, 'lcl_detune_ts':None}
    for i in range(2):
        values = []
        pulseName = pulse_shape_data.iloc[0,i]
        pulseParameter = pulse_parameter_data.iloc[0,i]
        pulseNoise = noise_data.iloc[i, 0]
        
        for t in times:
            values.append(pulse_dictionary[pulseName](time=t, pulse_parameter=pulseParameter, noise=pulseNoise))
        
        values[19] = 0.0
        values[0] = 0.0
        results[list(results.keys())[i]] = values
        

    times = times * micro_seconds
    
    # print(len(results['rabi_ts']))
    # print(len(np.array(results['rabi_ts'])*megahertz))
    # return times, results
    return times , TimeSeries.from_lists(times=times, values=np.array(results['rabi_ts'])*megahertz), TimeSeries.from_lists(times=times, values=np.array(results['lcl_detune_ts'])*megahertz)

In [766]:
# note, choose noise seed based on csv file
if noiseChoice == 2:
    noise = pd.read_csv(folder+"noise2.csv")
else:
    noise = pd.read_csv(folder+"noise.csv")

params = pd.read_csv(folder+"params.csv", index_col=False)
pattern = pd.read_csv(folder+"pattern.csv")
positions = pd.read_csv(folder+"positions.csv")
pulseNames = pd.read_csv(folder+"pulseNames.csv", index_col=False)



In [767]:
noise

,noise
0,0.250287
1,0.328475
2,0.575789


In [768]:
positions

,x,y
0,74.999900,44.512324
1,32.456927,63.075383
2,59.691486,43.955420
3,48.468808,52.916638


In [769]:
# Due to the limited spatial resolution of Aquila, it is sometimes necessary to slightly adjust the positions of the atoms by a small amount

if folder == 'pythonInput/MNIST_0/trapezoidPulsetrianglePulse/':
    positions["y"][0] = positions["y"][2]
elif folder == 'pythonInput/FashionMNIST_0/trapezoidPulsetrianglePulse/':
    separation = 2.0 - np.abs(positions["y"][2] - positions["y"][3])
    
    positions["y"][2] = positions["y"][2] + separation / 2.0
    positions["y"][3] = positions["y"][3] - separation / 2.0
elif folder == 'pythonInput/FashionMNIST_9/trapezoidPulseexpPulse/':
    separation = 2.0 - np.abs(positions["y"][2] - positions["y"][0])
    
    positions["y"][2] = positions["y"][2] - separation / 2.0
    positions["y"][0] = positions["y"][0] + separation / 2.0
elif folder == 'pythonInput/FashionMNIST_9/trianglePulsetrapezoidPulse/':
    separation = 2.0 - np.abs(positions["y"][2] - positions["y"][0])
    
    positions["y"][2] = positions["y"][2] + separation / 2.0
    positions["y"][0] = positions["y"][0] - separation / 2.0

elif folder == 'pythonInput/FashionMNIST_1/trapezoidPulsegaussianPulse/':
    positions["y"][0] = positions["y"][2]

elif folder == 'pythonInput/FashionMNIST_1/trapezoidPulselinearPulse/':
    
    positions["y"][3] = positions["y"][0]
    

/var/folders/l3/13yv8vgs5pz0dpt_4vcdbxvw0000gn/T/ipykernel_24472/1501864104.py:20: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  positions["y"][0] = positions["y"][2]


In [746]:
folder

'pythonInput/FashionMNIST_1/trapezoidPulselinearPulse/'

In [747]:
positions

,x,y
0,74.999900,74.204899
1,68.606876,50.048795
2,74.673674,59.638859
3,54.137528,74.204899


In [748]:
time, results_RF, results_lclDetune = getTimeSeries(pulseNames, noise, params, value_count=20)

In [752]:
from braket.ahs.hamiltonian import Hamiltonian
from braket.ahs.atom_arrangement import AtomArrangement
from braket.ahs.analog_hamiltonian_simulation import AnalogHamiltonianSimulation

register = AtomArrangement()
H = Hamiltonian()

ahs_program = AnalogHamiltonianSimulation(
    hamiltonian=H,
    register=register
)

In [753]:
register.add([positions.iloc[0,0] * micro_meter , positions.iloc[0,1] * micro_meter] )
register.add([positions.iloc[1,0] * micro_meter , positions.iloc[1,1] * micro_meter] )
register.add([positions.iloc[2,0] * micro_meter , positions.iloc[2,1] * micro_meter] )
register.add([positions.iloc[3,0] * micro_meter , positions.iloc[3,1] * micro_meter] )

In [754]:
from braket.ahs.driving_field import DrivingField

# e.g. all-zero phase and detuning
phi = TimeSeries().put(0.0, 0.0).put(t_max, 0.0)  # (time [s], value [rad])
Delta_global = TimeSeries().put(0.0, params.iloc[0, 3]*megahertz).put(t_max, params.iloc[0, 3]*megahertz)  # (time [s], value [rad/s])

drive = DrivingField(
    amplitude=results_RF,
    phase=phi,
    detuning=Delta_global
)

H += drive

In [755]:
from braket.ahs.field import Field
from braket.ahs.pattern import Pattern
from braket.ahs.local_detuning import LocalDetuning

# e.g. the local detuning only acts on the third atom, 
# which is the top atom in the triangular array.
h = Pattern(pattern['h'].to_list())

local_detuning = LocalDetuning(
    magnitude=Field(
        time_series=results_lclDetune,
        pattern=h
    )
)

H += local_detuning

In [756]:
from braket.aws import AwsDevice, AwsSession
from braket.devices import Devices
from pprint import pprint as pp

device = AwsDevice(Devices.QuEra.Aquila)
capabilities = device.properties.paradigm
# pp(capabilities.dict())

In [757]:
discretized_ahs_program = ahs_program.discretize(device)

In [758]:
ahs_program.to_ir()

Program(braketSchemaHeader=BraketSchemaHeader(name='braket.ir.ahs.program', version='1'), setup=Setup(ahs_register=AtomArrangement(sites=[[Decimal('0.0000749999'), Decimal('0.00007420489904109493')], [Decimal('0.00006860687632452519'), Decimal('0.000050048794539297046')], [Decimal('0.0000746736740602521'), Decimal('0.00005963885855330911')], [Decimal('0.000054137528091259605'), Decimal('0.00007420489904109493')]], filling=[1, 1, 1, 1])), hamiltonian=Hamiltonian(drivingFields=[DrivingField(amplitude=PhysicalField(time_series=TimeSeries(values=[Decimal('0.0'), Decimal('15799000.0'), Decimal('15799000.0'), Decimal('15799000.0'), Decimal('15799000.0'), Decimal('15799000.0'), Decimal('15799000.0'), Decimal('15799000.0'), Decimal('15799000.0'), Decimal('15799000.0'), Decimal('15799000.0'), Decimal('15799000.0'), Decimal('15799000.0'), Decimal('15799000.0'), Decimal('15799000.0'), Decimal('15799000.0'), Decimal('15799000.0'), Decimal('15799000.0'), Decimal('15799000.0'), Decimal('0.0')], time

In [759]:
n_shots = 1000

In [760]:
task = device.run(discretized_ahs_program, shots=n_shots)

In [713]:
positions

,x,y
0,74.999900,74.204899
1,68.606876,50.048795
2,74.673674,59.638859
3,54.137528,73.489248


The task ARN will let you retrieve results after they are run. Make sure to note which ARN corresponds to which class, pulse shape, and noise seed combination

In [ ]:
metadata = task.metadata()
task_arn = metadata['quantumTaskArn']
task_status = metadata['status']

print(f"ARN: {task_arn}")
print(f"status: {task_status}")